In [106]:
import nuclio 
import mlrun

In [107]:
%nuclio env -c V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env -c V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env -c V3IO_API=${V3IO_API}

In [108]:
%%nuclio cmd -c
pip install beautifulsoup4
pip install pandas
pip install v3io_frames

In [109]:
%%nuclio config 
kind = "nuclio"
spec.build.baseImage = "mlrun/ml-models"

%nuclio: setting kind to 'nuclio'
%nuclio: setting spec.build.baseImage to 'mlrun/ml-models'


In [110]:
%nuclio cmd -c pip install v3io

In [111]:
# nuclio: start-code

In [112]:
import mlrun.feature_store as fs
from mlrun.feature_store.steps import *
import mlrun
import pandas as pd
import numpy
import os

In [148]:
def handler(context, event):
    context.logger.info("Reading features from feature vector")
    stock_list = []
    # Reading the data from feature service
    service = fs.get_online_feature_service("stocks-vec")
    for key,value in context.sym_to_url.items():
        context.logger.info(key)
        context.logger.info(service.get([{"symbol": key}],as_list = True))
        data = service.get([{"symbol": key}])[0]
        data["symbol"] = key
        stock_list.append(data)
    
    data = [list(item.values()) for item in stock_list]
    
    columns = [{'text': key, 'type': 'object'} for key in stock_list[0].keys()]
    
    response = [{'columns': columns,
                'rows': data,
                'type': 'table'}]
    return response

In [149]:
def init_context(context):
    # Setting our enviornment
    context.logger.info("Initalizing context & constructing featrure vector")
    setattr(context, 'PROJECT_NAME', os.getenv('PROJECT_NAME', 'stocks-test-' + os.getenv('V3IO_USERNAME')))
    mlrun.set_environment(project=context.PROJECT_NAME)
    
    sym_to_url = {'GOOGL': 'google-inc', 'MSFT': 'microsoft-corp', 'AMZN': 'amazon-com-inc',
                  'AAPL': 'apple-computer-inc'}
    setattr(context, 'sym_to_url', sym_to_url)
    setattr(context, 'stocks_vec', "stocks-vec")
    
    # Declaring the features we would like to add to our vector
    features = ["stocks.prices_min_1h",
                "stocks.prices_max_1h",
                "stocks.volumes_min_1h",
                "stocks.volumes_max_1h",
                "stocks.price",
                "stocks.volume",
                "stocks.last_updated",
                "news.sentiments_min_1h",
                "news.sentiments_max_1h",
                "news.sentiment",
                "news.last_reaction"]
    
    # Making our vector
    vector = fs.FeatureVector(context.stocks_vec, features, description="stocks demo feature vector")
    vector.save()

In [150]:
# nuclio: end-code

In [151]:
# test local
init_context(context)
event = ""#nuclio.Event(body='')
df = handler(context,event)

Python> 2021-03-25 13:15:58,818 [info] Initalizing context & constructing featrure vector
Python> 2021-03-25 13:15:58,858 [info] Reading features from feature vector
Python> 2021-03-25 13:15:58,921 [info] GOOGL
Python> 2021-03-25 13:15:58,935 [info] [[inf, -inf, inf, -inf, 2033.22998046875, 29723.0, '2021-03-24 15:59:00-04:00', -3.3333333333333335, -3.3333333333333335, -3.333333333333333, '2021-03-25 12:50:55']]
Python> 2021-03-25 13:15:58,939 [info] MSFT
Python> 2021-03-25 13:15:58,943 [info] [[inf, -inf, inf, -inf, 235.4600067138672, 496721.0, '2021-03-24 15:59:00-04:00', 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, '2021-03-25 12:50:55']]
Python> 2021-03-25 13:15:58,947 [info] AMZN
Python> 2021-03-25 13:15:58,951 [info] [[inf, -inf, inf, -inf, 3087.169921875, 64163.0, '2021-03-24 15:59:00-04:00', -2.6666666666666665, -1.6666666666666667, -1.666666666666667, '2021-03-25 12:55:22']]
Python> 2021-03-25 13:15:58,955 [info] AAPL
Python> 2021-03-25 13:15:58,963 [info] [[inf

In [143]:
# test remote deployment
from mlrun import code_to_function
fn = code_to_function('vector_reader',
                      handler='handler')

# Set parameters for current deployment
fn.set_envs({'PROJECT_NAME' : 'stocks-test-dani',
             'STOCKS_STREAM':  os.getenv('V3IO_USERNAME') +' /stocks/stocks_stream'})
fn.spec.max_replicas = 2

In [144]:
addr = fn.deploy(project='stocks-test-dani')

> 2021-03-25 13:13:29,982 [info] Starting remote function deploy
2021-03-25 13:13:30  (info) Deploying function
2021-03-25 13:13:30  (info) Building
2021-03-25 13:13:30  (info) Staging files and preparing base images
2021-03-25 13:13:30  (info) Building processor image
2021-03-25 13:13:31  (info) Build complete
> 2021-03-25 13:13:35,488 [info] function deployed, address=default-tenant.app.dev8.lab.iguazeng.com:32364


In [145]:
!curl {addr}

[{"columns": [{"text": "sentiments_min_1h", "type": "object"}, {"text": "sentiments_max_1h", "type": "object"}, {"text": "prices_min_1h", "type": "object"}, {"text": "prices_max_1h", "type": "object"}, {"text": "volumes_min_1h", "type": "object"}, {"text": "volumes_max_1h", "type": "object"}, {"text": "price", "type": "object"}, {"text": "volume", "type": "object"}, {"text": "last_updated", "type": "object"}, {"text": "sentiment", "type": "object"}, {"text": "last_reaction", "type": "object"}, {"text": "symbol", "type": "object"}], "rows": [[-3.3333333333333335, -3.3333333333333335, Infinity, -Infinity, Infinity, -Infinity, 2033.22998046875, 29723.0, "2021-03-24 15:59:00-04:00", -3.333333333333333, "2021-03-25 12:50:55", "GOOGL"], [0.3333333333333333, 0.3333333333333333, Infinity, -Infinity, Infinity, -Infinity, 235.4600067138672, 496721.0, "2021-03-24 15:59:00-04:00", 0.3333333333333333, "2021-03-25 12:50:55", "MSFT"], [-2.6666666666666665, -1.6666666666666667, Infinity, -Infinity, In